### Bloco 1 - Configuração do Spark com suporte ao Delta lake

In [ ]:
import pyspark
from delta import *

In [ ]:
builder = pyspark.sql.SparkSession.builder \
    .appName("TestePraticoJordan") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

### Bloco 2 - Leitura do arquivo

In [ ]:
PATH = 'work'
FILENAME = 'financial_data.csv'

In [ ]:
df = spark.read.format('csv').option('header', True).load(f'{PATH}/{FILENAME}')

### Bloco 3 - Convertendo coluna "valor" para double 

In [ ]:
df = df.withColumn('valor', col('valor').cast('double'))

### Bloco 4 - Filtrando transações com valores acima de 1000

In [ ]:
df = df.filter(col('valor') > 1000)

### Bloco 5 - Salvando dados particionados por data transação em Delta Lake

In [ ]:
OUTPUT_PATH = '/home/jovyan/work/delta/financial_data'

In [ ]:
df.write \
    .format('delta') \
    .partitionBy('data_transacao') \
    .save(OUTPUT_PATH)

### Bloco 6 - Criando delta table a partir dos dados salvos

In [ ]:
INPUT_PATH = OUTPUT_PATH

In [ ]:
financial_data = DeltaTable.forPath(spark, INPUT_PATH)